In [1]:
# -*- coding: utf-8 -*-
"""GridSearchCV_PlantCLEF2013.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1aZBAmZMLizQ6dA9POQHLiQ-a7AHY0Fxc
"""

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
import time
import collections
from collections import defaultdict

# Commented out IPython magic to ensure Python compatibility.
#from google.colab import drive
#drive.mount('/gdrive/')
#%cd /gdrive/My\ Drive/artigo_plantas
#!ls


In [2]:
def multi_clf_metrics(arq_csv):

	# Cria X (features) e y (labels)
  especies = pd.read_csv(arq_csv, low_memory=False)
  X = np.array(especies.iloc[2:,0:4096])
  y = np.array(especies.iloc[2:,4096])		
  # Executa o PCA
  pca = PCA(.90)
  principal_components = pca.fit_transform(X)
 # 'SVC - (linear, ovr)': OneVsRestClassifier(svm.SVC(kernel='linear', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovr', random_state=None)),					'SVC - (linear, ovo)': svm.SVC(kernel='linear', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None),					'SVC - (polynomial, ovo)': svm.SVC(kernel='poly', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None),
#					'SVC - (sigmoid, ovr)': OneVsRestClassifier(svm.SVC(kernel='sigmoid', degree=3,coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovr', random_state=None)),
#					'SVC - (sigmoid, ovo)': svm.SVC(kernel='sigmoid', degree=3, coef0=0.0,
#											shrinking=True, probability=False, tol=0.001, cache_size=500,
#											class_weight=None, verbose=False, max_iter=-1,
#											decision_function_shape='ovo', random_state=None)
  classifiers = {
					'SVC - (polynomial, ovr)': OneVsRestClassifier(svm.SVC(kernel='poly', degree=3, coef0=0.0,
											shrinking=True, probability=False, tol=0.001, cache_size=2000,
											class_weight=None, verbose=False, max_iter=-1,
											decision_function_shape='ovr', random_state=None))
}

  scoring = ['f1_micro']
  # Parametros para rodar com o grid search
  parameters = {"estimator__C" : [0.01, 0.1, 10], 'estimator__gamma':[0.0001, 0.001, 0.01]}
  results = []
  f = open('gd_srch_results_'+arq_csv,'w')
  f.write('classifier name,'+'best parameters,'+'all results\n')
  f.close()
  for name_clf, clf in classifiers.items():
    print('Executando classificador'+ name_clf)
    scores = GridSearchCV(clf, parameters,cv=5, scoring=scoring, refit='f1_micro', return_train_score=True, n_jobs=-1, verbose=51)
    result = scores.fit(principal_components,y)
    results.append(result)
#    print('Melhor configuração: '+result.best_params_)
#    f = open('gd_srch_results_'+arq_csv,'w')
#    f.write(name_clf+',')
#    f.write(result.best_params_+',')
#    f.write(result+'\n')
#    f.close()
  return result

In [3]:
results = multi_clf_metrics('plantclef2013_vgg16.csv')


Executando classificadorSVC - (polynomial, ovr)
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Memmapping (shape=(26077, 527), dtype=float64) to new file /var/folders/k2/7m7wfg6n6y5fylg_14w25lnc0000gp/T/joblib_memmapping_folder_9244_4927505988/9244-115100386920-fdf5f14b40d941dfacfea4f6b6ea1465.pkl
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 35.5min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 35.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 35.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 35.9min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 105.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 106.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 107.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 143.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elaps

In [12]:
f = open('gd_srch_results_plantclef2013_vgg16.csv','w')
f.write(results.best_params_',')
f.write(results+'\n')
f.close()

SyntaxError: invalid syntax (<ipython-input-12-f5c1aa2caace>, line 2)

In [10]:
print(results.best_params_)

{'estimator__C': 0.01, 'estimator__gamma': 0.01}
